# 2.5 머신러닝 알고리즘을 위한 데이터 준비

## 2.5.0 준비

In [59]:
# 데이터 추출 함수
import os
import tarfile
import urllib

DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
HOUSING_PATH = '../' + os.path.join('datasets', 'housing')
HOUSING_URL = DOWNLOAD_ROOT + 'datasets/housing/housing.tgz'

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    현재 작업 공간에 "datasets/housing" 디렉터리 생성
    housing.tgz 파일 내려받음
    같인 디렉터리에 압축을 풀어 "housing.csv" 파일 생성
    """
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
#fetch_housing_data(HOUSING_URL, HOUSING_PATH)

In [60]:
# 데이터 읽기 함수
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [61]:
# 테스트 세트 분리
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# income_cat 특성 생성
housing['income_cat'] = pd.cut(housing['median_income'],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
# income_cat 특성 삭제
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)
    
print('strat_train_set size : {}'.format(len(strat_train_set)))
print('strat_test_set size : {}'.format(len(strat_test_set)))

strat_train_set size : 16512
strat_test_set size : 4128


<br>

**cf) 머신러닝 알고리즘을 위한 데이터 준비 작업을 자동화해야 하는 이유**

- 어떤 데이터셋에 대해서도 데이터 변환을 손쉽게 반복할 수 있어야 한다.  
(ex. 다음번에 새로운 데이터셋을 사용할 때)
- 향후 프로젝트에 사용할 수 있는 변환 라이브러리를 점진적으로 구축하게 된다.
- 실제 시스템에서 알고리즘에 새 데이터를 주입하기 전에 변환시키는 데 이 함수를 사용할 수 있다.
- 여러 가지 데이터를 변환을 쉽게 시도해볼 수 있고 어떤 조합이 가장 좋은 지 확인하는 데 편리하다.

In [62]:
# 예측 변수와 레이블 분리
# - drop()은 데이터 복사본을 만들며 strat_train_set에 영향을 주지 않음
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

<br>

## 2.5.1 데이터 정제

### 2.5.1.1 결측치 처리

- 대부분의 머신러닝 알고리즘은 누락된 특성을 다루지 못함
- 이를 처리할 수 있는 함수 생성

In [63]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
4629,-118.30,34.07,18.0,3759.0,NaN,3296.0,1462.0,2.2708,<1H OCEAN
6068,-117.86,34.01,16.0,4632.0,NaN,3038.0,727.0,5.1762,<1H OCEAN
17923,-121.97,37.35,30.0,1955.0,NaN,999.0,386.0,4.6328,<1H OCEAN
13656,-117.30,34.05,6.0,2155.0,NaN,1039.0,391.0,1.6675,INLAND
19252,-122.79,38.48,7.0,6837.0,NaN,3468.0,1405.0,3.1662,<1H OCEAN


- `total_bedrooms` 특성에 값이 없는 경우가 있음
- 이를 처리하는 방법은 3가지가 있음
- 데이터프레임의 `dropna()`, `drop()`, `fillna()` 메서드를 이용해 이런 작업을 할 수 있음

<br>

**1. 해당 구역을 제거**

```python
df.dropna(subsets=['total_bedrooms'])
```

In [64]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity


<br>

**2. 전체 특성을 삭제**

```python
df.drop('total_bedrooms', axis=1)
```

In [65]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)

,longitude,latitude,housing_median_age,total_rooms,population,households,median_income,ocean_proximity
4629,-118.30,34.07,18.0,3759.0,3296.0,1462.0,2.2708,<1H OCEAN
6068,-117.86,34.01,16.0,4632.0,3038.0,727.0,5.1762,<1H OCEAN
17923,-121.97,37.35,30.0,1955.0,999.0,386.0,4.6328,<1H OCEAN
13656,-117.30,34.05,6.0,2155.0,1039.0,391.0,1.6675,INLAND
19252,-122.79,38.48,7.0,6837.0,3468.0,1405.0,3.1662,<1H OCEAN


<br>

**3. 어떤 값으로 채움 (ex. 0, 평균, 중간값 등)**

```python
median = df['total_bedrooms'].median() # 중간값
df['total_bedrooms'].fillna(median, inplace=True)
```

In [66]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True)
sample_incomplete_rows

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
4629,-118.30,34.07,18.0,3759.0,433.0,3296.0,1462.0,2.2708,<1H OCEAN
6068,-117.86,34.01,16.0,4632.0,433.0,3038.0,727.0,5.1762,<1H OCEAN
17923,-121.97,37.35,30.0,1955.0,433.0,999.0,386.0,4.6328,<1H OCEAN
13656,-117.30,34.05,6.0,2155.0,433.0,1039.0,391.0,1.6675,INLAND
19252,-122.79,38.48,7.0,6837.0,433.0,3468.0,1405.0,3.1662,<1H OCEAN


<br>

- 계산한 값(ex. 중간값)은 다음과 같은 상황에 활용하기 위해 저장해 놓아야 한다.
  - 시스템을 평가할 때 테스트 세트에 있는 누락된 값을 채울 때 필요
  - 시스템이 실제 운영될 때 새로운 데이터에 있는 누락된 값을 채울 때 필요
- 테스트 세트의 중간값을 사용하면 훈련 세트에서 학습한 것이 소용없어지므로 사용하면 안된다.

<br>

### 2.5.1.2 `SimpleImputer`를 통한 결측치 처리

- 사이킷런의 `SimpleImputer`는 누락된 값을 손쉽게 다루도록 한다.

In [67]:
from sklearn.impute import SimpleImputer

# 누락된 값을 특성의 중간값으로 대체한다고 지정하여 객체 생성
imputer = SimpleImputer(strategy='median')

# 중간값이 수치형 특성에서만 계산될 수 있기 때문에 텍스트 특성인 ocean_proximity를 제외한 데이터 복사본 생성
housing_num = housing.drop('ocean_proximity', axis=1)

# imputer 객체의 fit() 메서드를 사용해 훈련 데이터에 적용
imputer.fit(housing_num)

# imputer는 각 특성의 중간값을 계산해서 그 결과를 객체의 statistics_ 속성에 저장
# total_bedrooms 특성에만 누락된 값이 있음
# 하지만 나중에 시스템이 서비스될 때 새로운 데이터에서 어떤 값이 누락될지 알 수 없음
# 그러므로 모든 수치형 특성에 imputer를 적용하는 것이 바람직함
print('imputer median : ', imputer.statistics_)
print('df calc median : ', housing.median().values)

# 학습된 imputer 객체를 사용해 훈련 세트에서 누락된 값을 학습한 중간값으로 바꿀 수 있음
X = imputer.transform(housing_num)

# 위의 실행 결과는 변형된 특성들이 들어 있는 평범한 넘파이 배열임
# 이를 다시 판다스 데이터프레임으로 간단히 되돌릴 수 있음
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

imputer median :  [-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]
df calc median :  [-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]


In [68]:
housing_tr.loc[sample_incomplete_rows.index.values]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
4629,-118.30,34.07,18.0,3759.0,433.0,3296.0,1462.0,2.2708
6068,-117.86,34.01,16.0,4632.0,433.0,3038.0,727.0,5.1762
17923,-121.97,37.35,30.0,1955.0,433.0,999.0,386.0,4.6328
13656,-117.30,34.05,6.0,2155.0,433.0,1039.0,391.0,1.6675
19252,-122.79,38.48,7.0,6837.0,433.0,3468.0,1405.0,3.1662


In [69]:
housing_tr.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347


In [70]:
housing_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 8 columns):
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16512 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
dtypes: float64(8)
memory usage: 1.8 MB


<br>

### cf) 사이킷런 설계 철학

**일관성**

- 모든 객체가 일관되고 단순한 인터페이스를 공유

- **추정기(estimator)**
  - 데이터셋을 기반으로 일련의 모델 파라미터들을 추정하는 객체를 추정기라 함
    - ex) `imputer` 객체
  - 추정 자체는 `fit()` 메서드에 의해 수행됨
  - 하나의 매개변수로 하나의 데이터셋만 전달
    - 지도 학습 알고리즘에서는 매개변수가 두 개(두 번째 데이터셋은 레이블을 담고 있음)
  - 추정 과정에서 필요한 다른 매개변수들은 모두 하이퍼 파라미터로 간주되고, *인스턴스 변수*로 저장됨  
  (보통 *생성자의 매개변수*로 전달)
    - ex) `imputer` 객체의 `strategy` 매개변수
    - *인스턴스 변수*
      - 객체지향 프로그래밍에서 객체가 각각 독립적으로 가지고 있는 변수
    - *생성자의 매개변수*
      - 파이썬 객체 생성 시 전달하는 매개변수를 의미
      - 이 매개변수는 파이썬에서 새로운 객체가 생성될 때 자동으로 포출되는 특수 메서드인 `__init__`에 전달됨
      - 객체 생성 시 호출되는 또 다른 특수 메서드인 `__new__`도 있지만 종종 `__init__`을 생성자라고 부름  
  
  
- **변환기(transformer)**
  - 데이터셋을 변환하는 추정기를 변환기라고 함
    - ex) `imputer`
  - 변환은 데이터셋을 매개변수로 전달받은 `transform()` 메서드가 수행함
  - 변환기는 변환된 데이터셋을 반환함
  - 이런 변환은 일반적으로 `imputer`의 경우와 같이 학습된 모델 파라미터에 의해 결정됨
    - `imputer`가 실제로 계산한 것 : 데이터셋에 있는 각 특성의 중간값
    - 사이킷런에서는 변환기도 추정기와 인터페이스가 같기 때문에 학습한다고 표현함
  - 모든 변환기는 `fit()`과 `transform()`을 연달아 호출하는 것과 동일한 `fit_transform()` 메서드를 가지고 있음
    - `fit_transform()`이 최적화되어 있어서 더 빠름  
  
  
- **예측기(predictor)**
  - 일부 추정기는 주어진 데이터셋에 대해 예측을 만들 수 있음
    - ex) `LinearRegression` 모델
  - 예측기의 `predict()` 메서드는 새로운 데이터셋을 받아 이에 상응하는 예측값을 반환함
  - 또한 테스트 세트(지도 학습 알고리즘이라면 레이블도 함께)를 사용해 예측의 품질을 측정하는 `score()` 메서드를 가짐
    - 어떤 예측기는 예측의 확신을 측정하는 메서드도 제공함
      - 분류 모델이 제공하는 메서드들(`predict_proba()`, `decision_function()`)
      
<br>

**검사 기능**

- 모든 추정기의 하이퍼파라미터는 공개(public) 인스턴스 변수로 직접 접근할 수 있음
  - ex) `imputer.strategy`
- 모든 추정기의 학습된 모델 파라미터도 접미사로 밑줄(`_`)을 붙여서 공개 인스턴스 변수로 제공됨
  - ex) `imputer.statistics_`

<br>

**클래스 남용 방지**

- 데이터셋을 별도의 클래스가 아니라 넘파이 배열이나 사이파이 희소(sparse) 행렬로 표현함
- 하이퍼파라미터는 보통의 파이썬 문자열이나 숫자임

<br>

**조합성**

- 기존의 구성요소를 최대한 재사용함
  - ex) 여러 변환기를 연결한 다음 마지막에 추정기 하나를 배치한 `Pipeline` 추정기를 쉽게 만들 수 있음

<br>

**합리적인 기본값**

- 사이킷런은 일단 돌아가는 기본 시스템을 빠르게 만들 수 있도록 대부분의 매개변수에 합리적인 기본값을 지정해둠

<br>

## 2.5.2 텍스트와 범주형 특성 다루기

### 2.5.2.1 범주형 특성의 데이터 확인

In [71]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

,ocean_proximity
17606,<1H OCEAN
18632,<1H OCEAN
14650,NEAR OCEAN
3230,INLAND
3555,<1H OCEAN
19480,INLAND
8879,<1H OCEAN
13685,INLAND
4937,<1H OCEAN
4861,<1H OCEAN


<br>

- 위 값들은 임의의 텍스트가 아니라 가능한 값을 제한된 개수로 나열한 것임
- 각 값은 카테고리를 나타냄
- 따라서 이 특성은 범주형 특성임을 알 수 있음

<br>

### 2.5.2.2 범주형 특성 데이터 숫자 변환

- 대부분의 머신러닝 알고리즘은 숫자를 다룸
- 따라서 위의 카테고리를 텍스트에서 숫자로 변환
- 사이킷런의 `OrdinalEncoder` 클래스 사용
  - 사이킷런 0.20 버전에 추가된 클래스
  - 하위 버전 사용 시 `pandas.Series.factorize()` 를 대신 사용

In [72]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

array([[0.],
       [0.],
       [4.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.]])

<br>

- `categories_` 인스턴스 변수를 사용해 카테고리 목록을 얻을 수 있음
- 범주형 특성마다 카테고리들의 1D 배열을 담은 리스트가 반환됨
  - 이 경우는 범주형 특성이 하나만 있으므로 배열 하나를 담은 리스트가 반환됨

In [73]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

<br>

### 2.5.2.3 위 방식의 한계

- 이 표현 방식의 문제는 머신러닝 알고리즘이 가까이 있는 두 값이 떨어져 있는 두 값보다 더 비슷하다고 생각한다는 점이다.
- 이 문제는 일반적으로 **카테고리별 이진 특성**을 만들어 해결한다.
  - 카테고리가 `<1H OCEAN` 일 때 한 특성이 `1`이고 그 외 특성은 `0`
  - 카테고리가 `INLAND` 일 때 한 특성이 `1`이고 그 외 특성은 `0`

<br>

### 2.5.2.4 원-핫 인코딩 (one-hot encoding)

- 한 특성만 `1`이고(핫) 나머지는 `0`으로 변환하는 것
- 새로운 특성을 **더미(dummy)** 특성이라도고 부름
- 사이킷런은 범주의 값을 원-핫 벡터로 바꾸기 위한 `OneHotEncoder` 클래스를 제공
  - 사이킷런 0.20 버전 이전에는 이 메서드가 정수 범주형 값만 인코딩할 수 있음
  - 0.20 버전부터 텍스트 범주형 값을 포함하여 다른 입력 타입도 처리할 수 있음

In [74]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

<br>

- 출력이 넘파이 배열이 아닌 사이파이(SciPy)의 **희소 행렬(sparse matrix)** 이다.
- 이는 수천 개의 카테고리가 있는 범주형 특성일 경우 매우 효율적이다.

- 이런 특성을 원-핫 인코딩하면 열이 수천 개인 행렬로 변하고, 각 행은 1이 하나뿐이고 그 외에는 모두 0으로 채워져 있음
- 0을 모두 메모리에 저장하는 것은 낭비이므로 희소 행렬은 0이 아닌 원소의 위치만 저장함
- 이 행렬을 거의 일반적인 2차원 배열로 사용할 수 있지만 (밀집된) 넘파이 배열로 바꾸려면 `toarray()` 메서드를 호출하면 됨
  - `housing_cat_1hot`은 사이파이가 지원하는 희소 행렬 중 행을 압축하는 `scr_matrix` 임

In [75]:
housing_cat_1hot.toarray()

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

<br>

- `OneHotEncoder` 객체 생성 시 `sparse=False`를 지정하면 희소 행렬이 아닌 넘파이 배열을 반환한다.

In [76]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

<br>

- 인코더의 `categories_` 인스턴스 변수를 사용해 카테고리 리스트를 얻을 수 있음

In [77]:
cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

<br>

### 2.5.2.5 그 외 범주형 특성 변환 방법

- 카테고리 특성이 담을 수 있는 카테고리 수가 많다면(ex. 국가 코드, 직업, 생물 종류 등) 원-핫 인코딩은 많은 수의 입력 특성을 만듬
- 이는 훈련을 느리게 하고 성능을 감소시킬 수 있음  
  
  
- 이런 현상이 나타나면 범주형 입력값을 이 특성과 관련된 숫자형 특성으로 바꾸고 싶을 것이다.
  - ex) `ocean_proximity` 특성을 해안까지 거리로 바꿀 수 있음
  - ex) 국가 코드 -> 국가의 인구, 1인당 GDP  
  
  
- 또는 각 카테고리를 **임베딩(embedding)**이라고 부르는 학습 가능한 저차원 벡터로 바꿀 수 있다.
- 훈련하는 동안 각 카테고리의 표현이 학습됨
- 이는 **표현 학습(representation learning)**의 한 예이다.
- Ch13, Ch17 참고

<br>

## 2.5.3 나만의 변환기

### 2.5.3.1 나만의 변환기 만드는 방법

- 사이킷런은 (상속이 아닌) **덕 타이핑(duck typing, 상속이나 인터페이스 구현이 아니라 객체의 속성이나 메서드가 객체의 유형을 결정하는 방식)**을 지원하므로 다음 3가지 메서드를 구현한 파이썬 클래스를 만들면 된다.
  - `fit()` (`self` 반환)
  - `transform()`
  - `fit_transform()`  
  
  
- 마지막 메서드는 `TransformerMixin`을 상속하면 자동으로 생성된다.
  - 파이썬에서 이름에 `Mixin` 이 있으면 객체의 기능을 확장하려는 목적으로 만들어진 클래스를 나타낸다.
  - `TransformerMixin`은 `fit_transform()` 메서드 하나를 가지고 있으며 이를 상속하는 모든 파이썬 클래스에 이 메서드를 제공한다.
  - `TransformerMixin`의 `fit_transform()` 메서드는 단순히 `fit()` 과 `transform()`을 **메서드 연결(method chaining)**한 것이다.  
  
  
- 또한 `BaseEstimator`를 상속하면(그리고 생성자에 `*args` 나 `**kargs` 를 사용하지 않으면) 하이퍼파라미터 튜닝에 필요한 두 메서드(`get_params()`, `set_params()`)를 추가로 얻게 된다.
  - `get_params()`와 `set_params()` 함수는 사이킷런의 파이프라인과 그리드 탐색에 꼭 필요한 메서드이다.
  - 따라서 모든 추정기와 변환기는 `BaseEstimator`를 상속해야 한다.
  - 이 두 메서드는 생성자에 명시된 매개변수만을 참조하므로 `*args` 나 `**kargs` 는 사용할 수 없다.

<br>

### 2.5.3.2 조합 특성을 추가하는 간단한 변환기

- 이 변환기는 `add_bedrooms_per_room`  하이퍼파라미터 하나를 가짐
- 기본값은 `True`로 지정 (합리적인 기본값 지정 필요)
- 이 특성을 추가하는 것이 머신러닝 알고리즘에 도움이 될 지 안 될지 이 하이퍼파라미터로 쉽게 확인해볼 수 있다.

In [78]:
# 수동 생성 방법
# housing['rooms_per_household'] = housing['total_rooms']/housing['households']
# housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms']
# housing['population_per_household'] = housing['population']/housing['households']

In [79]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,<1H OCEAN
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,<1H OCEAN
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,NEAR OCEAN
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,INLAND
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,<1H OCEAN


In [80]:
from sklearn.base import BaseEstimator, TransformerMixin

# 데이터프레임에서의 열 인덱스
#rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
# 동적 처리
col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [housing.columns.get_loc(c) for c in col_names]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    
    def __init__(self, add_bedrooms_per_room = True): # *args 나 **kargs 가 아님
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self # 더 이상 할 일이 없음
    
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix] # 가구당 방 갯수
        population_per_household = X[:, population_ix] / X[:, households_ix] # 가구당 인구수
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix] # 방 당 화장실 갯수
            return np.c_[X, 
                         rooms_per_household,
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, 
                         rooms_per_household,
                         population_per_household]   
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)
#housing_extra_attribs = attr_adder.transform(housing.to_numpy())
print(housing_extra_attribs[:5])

[[-121.89 37.29 38.0 1568.0 351.0 710.0 339.0 2.7042 '<1H OCEAN'
  4.625368731563422 2.094395280235988]
 [-121.93 37.05 14.0 679.0 108.0 306.0 113.0 6.4214 '<1H OCEAN'
  6.008849557522124 2.7079646017699117]
 [-117.2 32.77 31.0 1952.0 471.0 936.0 462.0 2.8621 'NEAR OCEAN'
  4.225108225108225 2.0259740259740258]
 [-119.61 36.31 25.0 1847.0 371.0 1460.0 353.0 1.8839 'INLAND'
  5.232294617563739 4.135977337110481]
 [-118.59 34.23 17.0 6592.0 1525.0 4459.0 1463.0 3.0347 '<1H OCEAN'
  4.50580997949419 3.047846889952153]]


In [43]:
housing.iloc[0]

longitude               -121.89
latitude                  37.29
housing_median_age           38
total_rooms                1568
total_bedrooms              351
population                  710
households                  339
median_income            2.7042
ocean_proximity       <1H OCEAN
Name: 17606, dtype: object

In [44]:
housing_extra_attribs[0]

array([-121.89, 37.29, 38.0, 1568.0, 351.0, 710.0, 339.0, 2.7042,
       '<1H OCEAN', 4.625368731563422, 2.094395280235988], dtype=object)

<br>

- `housing_extra_attribs` 는 넘파이 배열이기 때문에 열 이름이 없다.
- `DataFrame`으로 복원하려면 다음과 같이 할 수 있다.

In [81]:
housing_extra_attribs = pd.DataFrame(housing_extra_attribs,
                                     columns=list(housing.columns)+['rooms_per_household', 'population_per_household'],
                                     index=housing.index)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
17606,-121.89,37.29,38,1568,351,710,339,2.7042,<1H OCEAN,4.62537,2.0944
18632,-121.93,37.05,14,679,108,306,113,6.4214,<1H OCEAN,6.00885,2.70796
14650,-117.2,32.77,31,1952,471,936,462,2.8621,NEAR OCEAN,4.22511,2.02597
3230,-119.61,36.31,25,1847,371,1460,353,1.8839,INLAND,5.23229,4.13598
3555,-118.59,34.23,17,6592,1525,4459,1463,3.0347,<1H OCEAN,4.50581,3.04785


<br>

## 2.5.4 특성 스케일링(feature scaling)

### 2.5.4.1 특성 스케일링이 필요한 이유

- 몇 가지 경우(ex. 트리 기반 알고리즘)를 제외하고 머신러닝 알고리즘은 입력 숫자 특성들의 스케일이 많이 다르면 잘 작동하지 않는다.
  - 주택 가격도 이에 해당
  - 전체 방 개수의 범위 : 6 ~ 39,320
  - 중간 소득의 범위 : 0 ~ 15
- 타깃값에 대한 스케일링은 일반적으로 불필요하다.

<br>

### 2.5.4.2 특성 스케일링 방법

- 모든 특성의 범위를 같도록 만들어 주는 방법
  - min-max 스케일링 (정규화(normalization))
  - 표준화(standardization)

<br>

### 2.5.4.3 min-max 스케일링

- 정규화(normalization)라고 부름
- 데이터들이 0~1 범위에 들도록 값을 이동하고 스케일을 조정하면 됨
- 데이터에서 최솟값을 뺀 후 최댓값과 최솟값의 차이로 나누면 됨
- 사이킷런에는 이에 해당하는 `MinMaxScaler` 변환기를 제공
  - 0~1 사이를 원하지 않는다면 `feature_range` 매개변수로 범위를 변경할 수 있음

<br>

### 2.5.4.4 표준화 (standardization)

- 먼저 평균을 뺀 후 표준 편차로 나누어 결과 분포의 분산이 1이 되도록 한다.
  - 평균을 빼므로 표준화를 하면 항상 평균이 0이다.
- 표준화는 min-max 스케일링과 달리 범위의 상한과 하한이 없어 어떤 알고리즘에서는 문제가 될 수 있다.  
(ex. 신경망은 종종 입력값의 범위로 0에서 1사이를 기대함)
- 그러나 표준화는 **이상치에 영향을 덜 받는다.**
  - ex) 중간 소득을 (잘못해서) 100 이라 입력한 구역을 가정
  - min-max 스케일링 : 0~15 사이의 모든 다른 값을 0 ~ 0.15로 만듬
  - 표준화 : 크게 영향을 받지 않음
- 사이킷런에는 표준화를 위한 `StandardScaler` 변환기가 있음

<br>

### 2.5.4.5 특성 스케일링 시 주의 사항

- 모든 변환기에서 스케일링은 (테스트 세트가 포함된) 전체 데이터가 아니고 **훈련 데이터에 대해서만 `fit()` 메서드를 적용**해야 한다.
- 그런 다음 훈련 세트와 테스트 세트(그리고 새로운 데이터)에 대해 `transform()` 메서드를 사용한다.

<br>

## 2.5.5 변환 파이프라인

### 2.5.5.1 `Pipeline` 클래스

- 수많은 변환 단계는 정확한 순서대로 실행되어야 한다.
- 사이킷런의 `Pipeline` 클래스를 사용하여 연속된 변환을 순서대로 처리할 수 있다.

In [82]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [83]:
housing_num_tr

array([[-1.15604281,  0.77194962,  0.74333089, ..., -0.31205452,
        -0.08649871,  0.15531753],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.21768338,
        -0.03353391, -0.83628902],
       [ 1.18684903, -1.34218285,  0.18664186, ..., -0.46531516,
        -0.09240499,  0.4222004 ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.3469342 ,
        -0.03055414, -0.52177644],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.02499488,
         0.06150916, -0.30340741],
       [-1.43579109,  0.99645926,  1.85670895, ..., -0.22852947,
        -0.09586294,  0.10180567]])

- `Pipeline` 은 연속된 단계를 나타내는 `이름/추정기` 쌍의 목록을 입력으로 받는다.
- 마지막 단계에는 변환기와 추정기를 모두 사용할 수 있고 그 외에는 모두 변환기여야 한다.
  - 즉, `fit_transform()` 메서드를 가지고 있어야 한다.
  - 또는 `fit()` 과 `transform()` 메서드만 가지고 있어도 된다.  
  
  
- `이름`은 고유하고 이중 밑줄 문자(`__`)를 포함하지 않는다면 무엇이든 상관 없다.
  - 추후 하이퍼파라미터 튜닝할 때 필요  
  
  
- 파이프라인의 `fit()` 메서드를 호출하면 모든 변환기의 `fit_transform()` 메서드를 순서대로 호출한다.
  - `fit_transform()` 메서드가 없다면 `fit()` 과 `transform()`을 차례로 호출한다.
- 한 단계의 출력을 다음 단계의 입력으로 전달한다.
- 마지막 단계에서는 `fit()` 메서드만 호출한다.  
  
  
- 파이프라인 객체는 마지막 추정기와 동일한 메서드를 제공한다.
  - 위 예에서는 마지막 추정기가 변환기 `StandardScaler` 이므로 파이프라인이 데이터에 대해 모든 변환을 순서대로 적용하는 `transform()` 메서드를 가지고 있다.
  - 또한 앞에서 사용한 `fit_transform()` 메서드도 가지고 있다.

<br>

### 2.5.5.2 `ColumnTransformer` 클래스

- 지금까지 범주형 열과 수치형 열을 각각 다루었다.
- 하나의 변환기로 각 열마다 적절한 변환을 적용하여 모든 열을 처리할 수 있다면 더 편리할 것이다.
- 사이킷런 0.20 버전에서 이런 기능을 위해 `ColumnTransformer`가 추가되었다.
- 이 클래스는 판다스 `DataFrame`과 잘 동작한다.

In [84]:
# 주택 가격 데이터 전체 변환 적용
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# 수치형 열 이름의 리스트와 범주형 열 이름의 리스트 만듬
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

In [85]:
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

In [86]:
housing_prepared.shape

(16512, 16)

- `ColumnTransformer` 클래스 객체 생성
  - 이 생성자는 튜플의 리스트를 받음
  - 각 튜플의 구성 : 이름, 변환기, 변환기가 적용될 열 이름(또는 인덱스)
  - 각 변환기를 적절한 열에 적용하고 그 결과를 두 번째 축을 따라 연결함 (변환기는 동일한 개수의 행을 반환해야 함)  
  
  
- `OneHotEncoder`는 희소 행렬을 반환
- `num_pipeline`은 밀집 행렬을 반환
- 희소 행렬과 밀집 행렬이 섞여 있을 때 `ColumnTransformer`는 최종 행렬의 밀집 정도(0이 아닌 원소의 비율)를 추정한다.
- 밀집도가 임곗값(기본적으로 `sparse_threshold=0.3` 이다.)보다 낮으면 희소 행렬을 반환한다.  
  
  
- 튜플에 변환기를 사용하는 대신 삭제하고 싶은 열이 있다면 `"drop"` 문자열로 지정
- 튜플에 변환을 적용하지 않은 열이 있다면 `"passthrough"`로 지정  
  
  
- 기본적으로 나열되지 않은 열은 삭제됨
- 하지만 이런 열을 다르게 처리하고 싶다면 `remainder` 하이퍼파라미터에 어떤 변환기(또는 `"passthrough"`)를 지정할 수 있음
  - `remainder` 파라미터의 기본값 = `"drop"`